In [3]:
import math
import random
import time

class GameBoard:
    def __init__(self):
        # Initialize game board with empty positions and no winner
        self.positions = [' ' for _ in range(9)]
        self.current_winner = None

    def display_board(self):
        # Display current state of the game board
        for row in [self.positions[i*3:(i+1)*3] for i in range(3)]:
            print('| ' + ' | '.join(row) + ' |')

    @staticmethod
    def display_board_positions():
        # Display board positions for reference
        number_board = [[str(i) for i in range(j*3, (j+1)*3)] for j in range(3)]
        for row in number_board:
            print('| ' + ' | '.join(row) + ' |')

    def available_moves(self):
        # Return list of available moves (indices with ' ')
        return [i for i, spot in enumerate(self.positions) if spot == ' ']

    def empty_squares(self):
        # Check if there are empty spots left on the board
        return ' ' in self.positions

    def make_move(self, square, letter):
        # Attempt to make a move on the board
        if self.positions[square] == ' ':
            self.positions[square] = letter
            if self.winner(square, letter):
                self.current_winner = letter
            return True
        return False

    def winner(self, square, letter):
        # Check if the latest move results in a win
        row_ind = square // 3
        row = self.positions[row_ind*3:(row_ind+1)*3]
        if all([spot == letter for spot in row]):
            return True

        col_ind = square % 3
        column = [self.positions[col_ind+i*3] for i in range(3)]
        if all([spot == letter for spot in column]):
            return True

        if square % 2 == 0:
            diagonal1 = [self.positions[i] for i in [0, 4, 8]]
            if all([spot == letter for spot in diagonal1]):
                return True
            diagonal2 = [self.positions[i] for i in [2, 4, 6]]
            if all([spot == letter for spot in diagonal2]):
                return True

        return False

class AIPlayer:
    def __init__(self, letter):
        # Initialize AI player with a specific letter ('X' or 'O')
        self.letter = letter

    def get_move(self, game):
        # Determine AI's move using minimax algorithm
        if len(game.available_moves()) == 9:
            square = random.choice(game.available_moves())
        else:
            square = self.minimax(game, 0, -math.inf, math.inf, True)['position']
        return square

    def minimax(self, state, depth, alpha, beta, maximizing_player):
        # Implement minimax algorithm to find the best move
        max_player = 'O'
        other_player = 'X'

        if state.current_winner == other_player:
            return {'position': None, 'score': 1 * (state.empty_squares() + 1) if other_player == max_player else -1 * (state.empty_squares() + 1)}
        elif not state.empty_squares():
            return {'position': None, 'score': 0}

        if maximizing_player:
            best = {'position': None, 'score': -math.inf}
        else:
            best = {'position': None, 'score': math.inf}

        for possible_move in state.available_moves():
            state.make_move(possible_move, max_player if maximizing_player else other_player)
            sim_score = self.minimax(state, depth + 1, alpha, beta, not maximizing_player)
            state.positions[possible_move] = ' '
            state.current_winner = None
            sim_score['position'] = possible_move

            if maximizing_player:
                if sim_score['score'] > best['score']:
                    best = sim_score
                alpha = max(alpha, sim_score['score'])
            else:
                if sim_score['score'] < best['score']:
                    best = sim_score
                beta = min(beta, sim_score['score'])

            if beta <= alpha:
                break

        return best

class HumanPlayer:
    def __init__(self, letter):
        # Initialize human player with a specific letter ('X' or 'O')
        self.letter = letter

    def get_move(self, game):
        # Prompt human player for a move
        valid_square = False
        val = None
        while not valid_square:
            square = input(self.letter + '\'s turn. Input move (0-8): ')
            try:
                val = int(square)
                if val not in game.available_moves():
                    raise ValueError
                valid_square = True
            except ValueError:
                print('Invalid square. Try again.')
        return val

def play_game(board, x_player, o_player, print_game=True):
    # Main function to start and play the Tic-Tac-Toe game
    if print_game:
        board.display_board_positions()

    letter = 'X'
    while board.empty_squares():
        if letter == 'O':
            square = o_player.get_move(board)
        else:
            square = x_player.get_move(board)

        if board.make_move(square, letter):
            if print_game:
                print(letter + f' makes a move to square {square}')
                board.display_board()
                print('')

            if board.current_winner:
                if print_game:
                    print(letter + ' wins!')
                return letter

            letter = 'O' if letter == 'X' else 'X'

        time.sleep(0.8)

    if print_game:
        print('It\'s a tie!')
    return 'T'

if __name__ == '__main__':
    # Initialize players and board
    x_player = HumanPlayer('X')  # Human player for 'X'
    o_player = AIPlayer('O')     # AI player for 'O'
    game_board = GameBoard()     # Game board instance

    # Start the game
    play_game(game_board, x_player, o_player, print_game=True)


| 0 | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |
X's turn. Input move (0-8): 1
X makes a move to square 1
|   | X |   |
|   |   |   |
|   |   |   |

O makes a move to square 4
|   | X |   |
|   | O |   |
|   |   |   |

X's turn. Input move (0-8): 2
X makes a move to square 2
|   | X | X |
|   | O |   |
|   |   |   |

O makes a move to square 0
| O | X | X |
|   | O |   |
|   |   |   |

X's turn. Input move (0-8): 4
Invalid square. Try again.
X's turn. Input move (0-8): 5
X makes a move to square 5
| O | X | X |
|   | O | X |
|   |   |   |

O makes a move to square 8
| O | X | X |
|   | O | X |
|   |   | O |

O wins!
